In [111]:
%load_ext autoreload
%autoreload 2
    
import json
import os
import time
import openai
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from typing import List, Dict, Tuple
from openai_interface import GPT_Turbo
import tiktoken
pd.set_option('display.max_rows', 400)
from prompt_templates import question_answering_prompt_series, context_block

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
print(question_answering_prompt_series)


Your task is to synthesize and reason over a series of transcripts of an interview between Tom Bilyeu and his guest(s).
After your synthesis, use the series of transcripts to answer the below question.  The series will be in the following format:

```
Show Summary: <summary>
Show Guest: <guest>
Transcript: <transcript>
```


Start Series:
```
{series}
```
Question:

{question}

Answer the question and provide reasoning if necessary to explain the answer.

If the context does not provide enough information to answer the question, then 

state that you cannot answer the question with the provided context.


Answer:



In [2]:
data_path = './data/impact_theory_data.json'
with open(data_path) as f:
    data = json.load(f)

In [64]:
openai.api_key = os.environ['OPENAI_API_KEY']
encode = tiktoken.encoding_for_model('gpt-3.5-turbo-0613')

In [65]:
gpt = GPT_Turbo(model='gpt-3.5-turbo-16k')

In [5]:
system_msg = 'You are an expert at creating summaries of podcast shows'

prompt = "Given a podcast episode transcript surrounded by triple backticks, and the guest of the show, write a summary of the episode between 500 and 750 tokens in length:\n\
Guest: {guest}\n\
```{transcript}```\
"

system_msg_count = len(encode.encode(system_msg))
prompt_count = len(encode.encode(prompt))
total_input = system_msg_count + prompt_count
total_input

55

In [89]:
def single_sum(num: int, prompt: str=prompt, temp: float=0.0):
    d = data[num]
    content = ' '.join(d['content'].split()[:6000])
    guest = d['guest']
    summary_prompt = prompt.format(guest=guest, transcript=content)
    try:
        response = gpt.get_completion_from_messages(prompt=summary_prompt, temperature=temp, system_message=system_msg)
        return response
    except Exception as e:
        return e

In [90]:
def update(num: int, response: str):
    data[num]['summary'] = response

## Sequential Processing

In [92]:
for num in range(161, 162):
    guest = data[num]['guest']
    resp = single_sum(num, temp=0.0)
    print(f'GUEST: {guest}')
    print()
    print(f'SUMMARY: {resp}')
    # update(num, resp)

GUEST: Gary Litefoot Davis

SUMMARY: In this episode of Impact Theory, host Tom Bilyeu interviews Gary Litefoot Davis, a multi-talented individual who has achieved success as a rapper, actor, author, entrepreneur, and political figure. Davis shares his unconventional journey to success, which began with his family experiencing financial collapse. Despite the hardships, Davis learned the value of hard work and resilience from his father and grandfather. He emphasizes the importance of taking responsibility for one's own success and setting aggressive timelines to achieve goals. Davis also discusses the need to lead by example and the power of staying focused and moving forward, even in the face of adversity. He encourages individuals to break down their goals into manageable steps and learn from successful people in their field. Davis believes in the importance of self-belief and self-worth, especially for Native Americans, and aims to inspire others through his music and storytelling. 

In [93]:
update(num, resp)

In [34]:
data[num]['summary'], data[num]['guest']

("digest everything that I've read now, I'm really struck by the idea that if we're evolving culturally, we need to be certain that we're not limiting ourselves based on previously held beliefs. So when I think about this, if there is something that could be designed into the system that would allow people to be more loosely connected, where they would have access to information that's otherwise quite difficult to come by, and that would lead to faster growth and development, if you're growing and evolving at a faster rate, rates of self-evolution, you're more likely to be able to solve problems. So I think consumer capitalism, and I'm going to speak pretty roughly here, but it fixates people on things that are, they're not purpose built to solve. So the reason that I think is this immersion in consumer capitalism is leading people away from their sense of purpose, and it's making them less happy, and it's putting them into a position where they're less capable of solving problems. So 

In [110]:
contents = [d['content'] for d in data]
sums = [d['summary'] for d in data]

In [112]:
for d in data:
    del d['content']

In [115]:
for i, d in enumerate(data):
    d['content'] = contents[i]

In [119]:
with open('final_copy.json', 'w') as f:
    json.dump(data, f)

## Batch Processing

In [84]:
for i, sum in enumerate(summaries[28:], 162):
    update(i, sum)

In [88]:
data[161]['guest']

'Gary Litefoot Davis'

In [60]:
summaries = []
token_count = 0
rate_limit = 80000
batch = 1
start = 134
pause = 30
for i, d in enumerate(tqdm(data[start:]), start):
    content = ' '.join(d['content'].split()[:5000])
    input_count = len(encode.encode(content))
    token_count += (input_count + total_input)
    # print(f'Total Token Count: {token_count}')
    if token_count > rate_limit * batch:
        print(f'Bumping against Token Limit sleeping for {pause} seconds...')
        time.sleep(pause)
        batch += 1
    guest = d['guest']
    summary_prompt = prompt.format(guest=guest, transcript=content)
    try:
        response = gpt.get_completion_from_messages(prompt=summary_prompt, system_message=system_msg)
        summaries.append(response)
        print(f'EPISODE: {i}')
        print(f'GUEST: {guest}')
        print(f'SUMMARY: {response}')
        print('\n\n')
    except Exception as e:
        print(e)
        summaries.append(f'Error: {e}')

  0%|          | 0/250 [00:00<?, ?it/s]

EPISODE: 134
GUEST: Tom Bilyeu
SUMMARY: In this podcast episode, Tom Bilyeu, co-founder of Impact Theory and Quest Nutrition, shares his insights on how to build an unbreakable mindset. He emphasizes the importance of taking ownership of one's life and not tolerating weakness. Bilyeu believes that life is tough, but individuals have a choice to be strong and take responsibility for their own success. He debunks the idea of a fixed mindset and encourages listeners to constantly seek ways to improve and learn. Bilyeu also discusses the cost of weakness, which includes a loss of self-respect and a lack of progress in life. On the other hand, he highlights the benefits of strength, such as earning respect and achieving fulfillment. Bilyeu provides concrete steps to develop an unbreakable mindset, including owning one's life, creating new values, building a value stack, and developing mental toughness. He emphasizes the importance of self-reliance, self-respect, growth, and resilience in th

In [61]:
len(summaries)

250

In [48]:
for i, sum in enumerate(data, 134):
    print()
    print(f'Guest: {data[i]["guest"]}')
    print(f'Episode #: {i}')
    print()
    print(f'Summary: {data[i]["summary"]}')
    print('-'*150)


Guest: Tom Bilyeu
Episode #: 134

Summary: In this episode, Tom Bilyeu, co-founder of Impact Theory and Quest Nutrition, shares his personal journey of overcoming emotional weakness and insecurity to achieve success. He emphasizes the importance of building a mindset that makes one unbreakable and unstoppable. Bilyeu discusses the cost of weakness, the value of strength, and the need to take complete ownership of one's life. He highlights the significance of developing a growth mindset, challenging fixed beliefs, and pursuing fulfillment over external success. Bilyeu also stresses the importance of self-reliance, self-respect, growth, and resilience as key values to become unstoppable. He encourages listeners to develop mental toughness, grit, and the ability to move towards fear. Throughout the episode, Bilyeu provides practical steps and strategies for individuals to take ownership of their lives and build the mindset needed to achieve their goals. He also offers a free workshop on 

KeyError: 'summary'

In [377]:
for i, num in enumerate(range(89,97)):
    print(i, num)
    data[num]['summary'] = summaries[i]

0 89
1 90
2 91
3 92
4 93
5 94
6 95
7 96


### Title Similarity Evaluation

In [6]:
titles = [d['title'] for d in data]

In [7]:
more_titles = titles * 3
len(more_titles)

1152

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('all-MiniLM-L6-v2')

In [9]:
embedded_titles = model.encode(titles)

In [10]:
def get_topk_titles(title: str, 
                    title_embeddings: np.array, 
                    titles: List[str], 
                    top_k: int=5,
                    titles_only: bool=True
                   ) -> List[Tuple[str, float]]: 
    
    title_embedding = np.expand_dims(model.encode(title), 0)
    scores = cosine_similarity(title_embedding, title_embeddings).squeeze().tolist()
    combined = list(zip(titles, scores))
    sorted_results = sorted(combined, key=lambda x: x[1], reverse=True)[:top_k]
    if titles_only:
        return [t[0] for t in sorted_results]
    else: return sorted_results

In [11]:
get_topk_titles('the effects of drugs on the brain', embedded_titles, titles)

['The Future Is Your Brain on Drugs | Jamie Wheal on Impact Theory',
 'The 6 Steps To BOOST BRAIN HEALTH & Reverse Cognitive DECLINE | Dr. Daniel Amen',
 'These DAILY HABITS Will Prime Your Brain To DESTROY Stress, Anxiety, & Depression | Tony Robbins',
 'How You Can Turn Addictive Behaviors into an Obsessive Focus | Dotsie Bausch on Impact Theory',
 'Neuroscientist Explains How Your Brain Is Affected by Fear, Isolation & Anxiety | Moriel Zelikowsky']

In [12]:
import tiktoken 
encoding = tiktoken.get_encoding("cl100k_base")
from prompt_templates import question_answering_prompt


def validate_token_threshold(ranked_results: List[dict], 
                             base_prompt: str,
                             query: str,
                             tokenizer: tiktoken.Encoding, 
                             token_threshold: int, 
                             verbose: bool = False
                             ) -> List[dict]:
        """
        Validates that prompt is below the set token threshold by adding lengths of:
            1. Base prompt
            2. User query
            3. Context material
        If threshold is exceeded, context results are reduced incrementally until the 
        combined prompt tokens are below the threshold. 
        """
        overhead_len = len(tokenizer.encode(base_prompt.format(question=query, context='')))
        context_len = get_batch_length(ranked_results, tokenizer)
    
        token_count = overhead_len + context_len
        if token_count > token_threshold:
            print('Token count exceeds token count threshold, reducing size of returned results below token threshold')
            
            while token_count > token_threshold and len(ranked_results) > 1:
                num_results = len(ranked_results)
                
                # remove the last ranked (most irrelevant) result
                ranked_results = ranked_results[:num_results-1]
                # recalculate new token_count
                token_count = overhead_len + get_batch_length(ranked_results, tokenizer)

        if verbose:
            print(f'Total Final Token Count: {token_count}')
        return ranked_results

def get_batch_length(ranked_results: List[dict], tokenizer: tiktoken.Encoding) -> int:
    contexts = tokenizer.encode_batch([r['content'] for r in ranked_results])
    context_len = sum(list(map(len, contexts)))
    return context_len

In [13]:
resp = [{'content': "And what is that spell? Culture? Family pressures? All of the above? Yes. If you look at the world, there you see where it got to. The world is polluting and it doesn't give a shit. The world is on fire. Nobody gives a shit. Only the people, the locals, but the rest of the world is just watching. War is going on. Watching. Refugees are drowning. Little children are drowning. We are watching. The production, that is important. Money, that is important. Competition, that is important. People die in the hospitals because of the COVID. They don't talk about how to strengthen your own immune system. Why? We have shown in 2014, Tom, we have shown how to suppress effectively with a group of people. So the scientific evidence that within a quarter of an hour, using specific breathing techniques to alter the immune system, to make the innate immune system go down, that means the cytokine storm, which is the damaging factor of COVID, of COVID-19 damaging factor, is inflammation.",
  'doc_id': 'n5j4d-OJcsw_19',
  'episode_url': 'https://www.youtube.com/watch?v=n5j4d-OJcsw&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Wim Hof',
  'length': 2969,
  'thumbnail_url': 'https://i.ytimg.com/vi/n5j4d-OJcsw/hq720.jpg',
  'title': 'Wim Hof “The Iceman” on the Resilience and Strength Your Body is Truly Capable of Achieving',
  'video_id': 'n5j4d-OJcsw',
  'views': 169225,
  'explainScore': '(bm25): original score 4.5607424, normalized score: 0.5',
  'score': '0.5'},
 {'content': "Why make them go so specific? Because when somebody tells me, we were sitting at the restaurant. We go on a date night once a month and the check came and he didn't offer to pick it up or she didn't offer to pick it up. I go, oh, okay. So what does that mean to you? Who normally picks up the check? What kind of restaurant were you at? And when they tell me, oh, you like sushi? I go, you like omakase? What were you guys eating? Sashimi? Oh, that's fancy. What are you guys? A coastal elite? I'm joking. I'm building rapport. I'm showing them that I'm not Mr. Money Guy with a suit and tie. Like I can speak their language, but I'm also trying to understand... I'm trying to meet them where they are. And to meet them where they are, I need to go into their target conversation or their sushi dinner. That's where people live. People do not normally live in a spreadsheet. They certainly don't live talking about money every day.",
  'doc_id': 'Yzvg0pWez2M_91',
  'episode_url': 'https://www.youtube.com/watch?v=Yzvg0pWez2M&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Ramit Sethi',
  'length': 6807,
  'thumbnail_url': 'https://i.ytimg.com/vi/Yzvg0pWez2M/hq720.jpg',
  'title': 'The Common MONEY TRAPS You Need To AVOID To Build Wealth | Ramit Sethi',
  'video_id': 'Yzvg0pWez2M',
  'views': 172976,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.32247633, normalized score: 0.5',
  'score': '0.5'},
 {'content': "My granddad, your grandfather. OK, my grandfather, there was no tax. Not it was back. Well, they didn't have income taxes. They didn't have Social Security taxes. You know, all the only taxes you had. It was like a tariff. You didn't even see the tariffs. Right. Because they were they were embedded in the products. And the cities, the locals, you know, they did have like sales tax. But that's it. That's all there was. Nobody had to fill out forms. Nobody had any money taking out of their paycheck. I mean, that's that's what America used to be like. And people weren't starving. You know, people were people were dying for lack of medical care. If you didn't have money, doctors worked for nothing. I mean, it was it was we ran great. Government was tiny. You know, 100 years ago, you know, 1900, 1890, 1910. You know that we didn't have as much technology then as we have now.",
  'doc_id': '_I8lzTePXQw_64',
  'episode_url': 'https://www.youtube.com/watch?v=_I8lzTePXQw&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Peter Schiff',
  'length': 10219,
  'thumbnail_url': 'https://i.ytimg.com/vi/_I8lzTePXQw/hq720.jpg',
  'title': 'A Recession Worse Than 2008? - How To Survive & Thrive The Next Economic Crisis | Peter Schiff',
  'video_id': '_I8lzTePXQw',
  'views': 391908,
  'explainScore': '(bm25): original score 4.169473, normalized score: 0.44667935',
  'score': '0.44667935'},
 {'content': "How are you going to hide your building? Buildings get property taxed. There's a very famous story about a bunch of luxury yachts sitting in Sardinian Port, and the locals decided that it wasn't fair that all these rich people were sitting on their yachts in the port spending all this money, but they weren't paying enough taxes. Now, they were putting millions and millions of euros into the economy, but they came up with the idea that they were going to put a tax on the yacht, on the value of the yacht, and so they passed a yacht tax that would have cost people millions or tens of millions of euros if they stayed in that port, and everything was happy, and all the restaurateurs and the hotelers and the entertainment people in the port, they were all happy making tons of money off the yachts until the day before the tax went into place, and the morning that the tax went into place, the port was empty and the economy died. Everybody left because yachts are floating capital. It just moves. It's floating property, right?",
  'doc_id': 'Ll5EDt13GcQ_85',
  'episode_url': 'https://www.youtube.com/watch?v=Ll5EDt13GcQ&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Michael Saylor',
  'length': 10592,
  'thumbnail_url': 'https://i.ytimg.com/vi/Ll5EDt13GcQ/hq720.jpg',
  'title': '"PREPARE NOW!" - This Is Your LAST CHANCE To Become A MILLIONAIRE In 2023 | Michael Saylor',
  'video_id': 'Ll5EDt13GcQ',
  'views': 457293,
  'explainScore': '(bm25): original score 4.037649, normalized score: 0.42871487',
  'score': '0.42871487'},
 {'content': "Do people just send you a shit ton of music now? Is that how you find people? No. It's a lot of it's like friends, networks of, like, I trust friends and advice from friends. That's probably been my go-to for signing the right artists on Denmark to, like, working and collabing with different artists. And plus, I get to travel the world. That helps me a lot. And I'm also putting myself out there. So when I go to a country, you know, I'm, like, putting out the feelers, like, can you tell, like, the artists in this country? I'm in Brazil, whatever I am. I want to hear, like, who's shaking up the noise. And the thing is, is that sometimes the biggest artists in that country is, like, unknown in my country, which happens a lot because they're singing entirely in Portuguese or Italian or whatnot. And that's what I did actually just recently. I was in Italy and my day-to-day, Dougie, who's in this room right now.",
  'doc_id': 'NHOOgWNaGwM_23',
  'episode_url': 'https://www.youtube.com/watch?v=NHOOgWNaGwM&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Steve Aoki',
  'length': 2813,
  'thumbnail_url': 'https://i.ytimg.com/vi/NHOOgWNaGwM/hq720.jpg',
  'title': "If You Can't Ditch Social Media Entirely, Try This Instead | Steve Aoki on Impact Theory",
  'video_id': 'NHOOgWNaGwM',
  'views': 170621,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.29196346, normalized score: 0.40301123',
  'score': '0.40301123'},
 {'content': "You really want to understand and get outside what's happening in the United States ecosystem. Watch the CBC or Al Jazeera or Deutsche Welle or NHK in Japan. Just watch their English language news once a week for half an hour, an hour. It's not very exciting, but it's like a completely external view of what the hell is going on in the United States and the rest of the world. And that forced you. First of all, it's long form, right? It's not the headlines beating you down. And secondly, it's like you don't actually have your anchor of all of the things that are stirring you up. They're not even playing with that. They're just kind of reporting on the best they can tell what the hell is going on. And then they're occasionally talking to people like that are locals and whatnot, but from every side that that's very valuable. But the thing that worries me about AI, I don't believe that AI is becoming much more like human beings. They're not faking us out by by just being it by being able to replicate me.",
  'doc_id': 'nXJBccSwtB8_28',
  'episode_url': 'https://www.youtube.com/watch?v=nXJBccSwtB8&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Ian Bremmer',
  'length': 5410,
  'thumbnail_url': 'https://i.ytimg.com/vi/nXJBccSwtB8/hq720.jpg',
  'title': "THE BIG AI RESET: The Next Global SuperPower Isn't Who You Think | Ian Bremmer",
  'video_id': 'nXJBccSwtB8',
  'views': 138628,
  'explainScore': '(bm25): original score 3.8403447, normalized score: 0.40182698',
  'score': '0.40182698'},
 {'content': "So I've worked in the inner cities a lot. And one thing I've noticed is that the inner city breaks almost everyone that it touches. But there are a few people that manage to be so transformed by it, whether it's out of I never want to go back, whether it's it just hardened them and they had to be smart to survive, whatever it is, there's a certain set of lessons that some people learn that then propel them so hard, so far, so fast out of the inner cities that they go on to accomplish just truly extraordinary things. Needless to say, put your name among those people. So walk us through, what were some of the key moments that you experienced as a kid? And how did you end up interpreting it in a way that pushed you forward, even if it was just I have to get the hell away from this? Yeah. I feel like there are a lot of moments where it's just like light bulb click. But I feel like one thing that I had that a lot of people don't have is even though I had a lot of experience seeing the stuff that happens in our communities, I also had access to the other side.",
  'doc_id': 'RXo4WSt9lc4_15',
  'episode_url': 'https://www.youtube.com/watch?v=RXo4WSt9lc4&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Tom Bilyeu',
  'length': 1708,
  'thumbnail_url': 'https://i.ytimg.com/vi/RXo4WSt9lc4/hq720.jpg',
  'title': '6 Experts Share Important Lessons on Being Grateful | Impact Theory',
  'video_id': 'RXo4WSt9lc4',
  'views': 73254,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.28782237, normalized score: 0.3898483',
  'score': '0.3898483'},
 {'content': "But what did you learn in the inner cities? I didn't really learn anything. I just knew I had to get out. When I sat in front of a television, man, I just saw things that I didn't see when I walked out my front door. I lived in a, it was two parents and nine kids living in a one bedroom shotgun house. If you don't know what a shotgun house is, you open the front door and you can shoot a gun out the back door and it doesn't hit anybody. So I know exactly where I grew up and I knew I knew exactly the environment that I saw on TV was not around here. I grew up. I rode a city bus. I didn't see a black person till I got downtown. TV is amazing to me, man, because, you know, information is amazing. I think that if you give people in inner city information, they will get out a lot faster. But what happens is if every one of my friends that I hung out with, if we all stayed in the inner city, I would not be sitting here.",
  'doc_id': '1QdMfSTMYoU_26',
  'episode_url': 'https://www.youtube.com/watch?v=1QdMfSTMYoU&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Rushion McDonald',
  'length': 2181,
  'thumbnail_url': 'https://i.ytimg.com/vi/1QdMfSTMYoU/hq720.jpg',
  'title': 'How to Reinvent Yourself At Any Age | Rushion McDonald on Impact Theory',
  'video_id': '1QdMfSTMYoU',
  'views': 1072887,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.2853554, normalized score: 0.3820067',
  'score': '0.3820067'},
 {'content': "Needs to bump their knee a little bit and understand that it's not that bad. Tom Bilyeu Yeah. I've heard you say that sometimes it's good to have your back against the wall. I have a theory about the inner cities, so I've worked in the inner cities a lot. One thing I've noticed is that the inner city breaks almost everyone that it touches, but there are a few people that manage to be so transformed by it, whether it's out of I never want to go back, whether it's it just hardened them and they had to be smart to survive, whatever it is, there's a certain set of lessons that some people learn that then propel them so hard, so far, so fast out of the inner cities that they go on to accomplish just truly extraordinary things. Needless to say, put your name among those people. Walk us through, what were some of the key moments that you experienced as a kid, and how did you end up interpreting it in a way that pushed you forward, even if it was just I have to get the hell away from this? Jullien Gordon Yeah.",
  'doc_id': 'ldgWOQvoZs8_14',
  'episode_url': 'https://www.youtube.com/watch?v=ldgWOQvoZs8&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Chamillionaire',
  'length': 2730,
  'thumbnail_url': 'https://i.ytimg.com/vi/ldgWOQvoZs8/hq720.jpg',
  'title': 'How This Famous Rapper Hustled to Become a Successful Investor | Chamillionaire on Impact Theory',
  'video_id': 'ldgWOQvoZs8',
  'views': 589240,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.28011912, normalized score: 0.3653626',
  'score': '0.3653626'},
 {'content': "The main difference is that it's physically distributed rather than concentrated in one place. So it might have the total land area of Estonia, collectively owned by the people, but just spread all over the world in ranches, and subdivisions, and towns, and cul-de-sacs. Like, imagine a China town was actually collectively owned by the people, and part of a global network of them, okay? And just to see how a network state could build out, do you see this GIF on screen? All right, so I'm just going to play this. It starts from one person in Tokyo, okay? If I play this, you go from one person in Tokyo, they're just making, you know, this is how a founder could go from one to a million, right? One, then 17 people, 172, 1,729, 17,000, 170,000, and so on. This is how the network state builds out, okay? And you essentially recruit people from around the world who have this collective elucidation, and they crowdfund larger and larger things until you start a new country. As crazy as that is, right?",
  'doc_id': 'l4fLax7S2Q0_227',
  'episode_url': 'https://www.youtube.com/watch?v=l4fLax7S2Q0&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Balaji Srinivasan',
  'length': 16255,
  'thumbnail_url': 'https://i.ytimg.com/vi/l4fLax7S2Q0/hq720.jpg',
  'title': 'EMERGENCY EPISODE: Why This Financial Crisis Is WORSE Than 2008 | Balaji Srinivasan',
  'video_id': 'l4fLax7S2Q0',
  'views': 528422,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.27507627, normalized score: 0.34933332',
  'score': '0.34933332'},
 {'content': "If competition shows up, I have to out-service them, out-work them, out-social media them versus shrivel up and die. So if you met someone who literally they're fresh off the boat here from somewhere, communist country, don't speak the language, you know what's in store for them. And I give you like an hour with them. What are like some key principles that you're going to teach them to endure? Ooh, great question. Thing number one is don't go find where the group of your people are. In other words, if my dad had come to Southern California and instead of living in Santa Ana, Anaheim, Garden Grove, that area, if we had moved to Glendale where there's a whole bunch of Armenians, we would not thrive like this. We would not thrive because it blows me away when immigrants come to this country and then they go to little Italy, little Armenia, little Saigon. Get the fuck out of little Saigon. Go to little white America. Learn the language. Assimilate. Understand the culture. Blend into the fucking fiber of this country because when you do, life becomes increasingly easier.",
  'doc_id': '0z2Kio2xlGI_21',
  'episode_url': 'https://www.youtube.com/watch?v=0z2Kio2xlGI&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Bedros Keuilian',
  'length': 2699,
  'thumbnail_url': 'https://i.ytimg.com/vi/0z2Kio2xlGI/hq720.jpg',
  'title': 'How to Build Success from Nothing | Bedros Keuilian on Impact Theory',
  'video_id': '0z2Kio2xlGI',
  'views': 579479,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.2669134, normalized score: 0.32338673',
  'score': '0.32338673'},
 {'content': "Tall poppy syndrome, what's that? It is basically a consciousness because Australia was a convict place, it's where England pushed all their convicts, this idea that the tallest poppy is the one that gets chopped down. You keep yourself sort of level with everyone else and nobody wants to be too successful or too boisterous or big because you will be chopped down by everybody in your community. That is actually a thing. It doesn't show up in the US. It's the exact opposite. You can have a Mercedes and live with your mom because you want to pretend like you're better than everyone else. Every place has its own thing. I just got back from London, England and their thing is the stiff upper lip, don't talk to me, I'm taking care of myself right now. All of those things don't work, they don't work. Tanzania, yes, everywhere has a thing. The degree from which they are receiving Western influence has a big part of it for me in my experience. Because they're not being bombarded with imagery that tells them that they're not good enough, community is still probably number one on the list.",
  'doc_id': '7O_yR9WVSFw_29',
  'episode_url': 'https://www.youtube.com/watch?v=7O_yR9WVSFw&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Preston Smiles',
  'length': 2447,
  'thumbnail_url': 'https://i.ytimg.com/vi/7O_yR9WVSFw/hq720.jpg',
  'title': 'How to Love the Process | Preston Smiles on Impact Theory',
  'video_id': '7O_yR9WVSFw',
  'views': 189840,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.25801206, normalized score: 0.2950927',
  'score': '0.2950927'},
 {'content': "In the context of what you just said, that was jarring for me just to hear you describe it about the human warmth, the human connection, and then to go to a place like Harvard in Western civilization and to feel more distant, more disconnected and ultimately then that becomes the beginning of the emotional distress that you go through. How do we reconcile that? What are your thoughts around the disconnect of the traditional Western lifestyle and the beauties that are in these rural villages that you're trying to really help? You're trying to help them with these Western ideals. It's pretty fascinating. Jessica Flanigan I know. It's such an odd juxtaposition because I find that some of my happiest moments are when I have very little and I'm in a place like rural Ghana or one of the places I love most in the world is rural northern Uganda, which is just stunning. It's pristine. I've had so many amazing experiences of just deep connection with the land but also with people. I think there's a certain vulnerability that comes from not having a lot of stuff. When you're poor, you depend on each other.",
  'doc_id': '4G-YmXS9_bs_44',
  'episode_url': 'https://www.youtube.com/watch?v=4G-YmXS9_bs&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Leila Janah',
  'length': 3202,
  'thumbnail_url': 'https://i.ytimg.com/vi/4G-YmXS9_bs/hq720.jpg',
  'title': 'From Cleaning Toilets to CEO, Leila Janah on How Rejection Is Inevitable & the Key to Success & Grit',
  'video_id': '4G-YmXS9_bs',
  'views': 186204,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.25110006, normalized score: 0.27312213',
  'score': '0.27312213'},
 {'content': "I'm almost certain that I have the exact quote here I Have molded my life around the fact that work is the best way to move people out of poverty and that work is at the Core of human dignity, but it's not all that there is so in the context of what you just said like That's that was jarring for me just to hear you describe it about the human warmth the human connection and then to go to a place Like Harvard in you know Western civilization and to feel more distant more disconnected and ultimately then sort of that becomes the beginning of The emotional distress that you go through like how do we reconcile that? like what are your thoughts around the the disconnect of The traditional Western lifestyle and the beauties that are in these rural villages that you're trying to to really help You're trying to help them with like these Western ideals. It's pretty fascinating I know and it's such an odd juxtaposition because you know I find that some of my happiest moments are when I have very little and I'm in a place like rural Ghana or one of The places I love most in the world is rural northern Uganda, which is just stunning.",
  'doc_id': '0gVTnivW2lE_50',
  'episode_url': 'https://www.youtube.com/watch?v=0gVTnivW2lE&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Leila Janah',
  'length': 3326,
  'thumbnail_url': 'https://i.ytimg.com/vi/0gVTnivW2lE/hq720.jpg',
  'title': 'The Keys to Building Character | Leila Janah on Impact Theory',
  'video_id': '0gVTnivW2lE',
  'views': 208730,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.24850553, normalized score: 0.2648751',
  'score': '0.2648751'},
 {'content': "No, this was in a hospital. And so I worked in a hospital in Arctic Alaska. And then I was a hospital in Arctic Alaska. Yeah, more than one, actually. Well, so there's one. So these are towns. These are towns. Yeah, I got it. I also worked on a cruise ship in the Russian Arctic that went to this island called Wrangell Island, which is sort of just northwest of the Bering Strait, has a lot of polar bears there. And it was totally spectacular. And so I was the doctor on that ship for about 50 people. And these boats are very well stocked. I was actually surprised. But every northern summer, there's a lot of cruise ships that are going through the Arctic, visiting places like Iceland and Greenland and Alaska and Svalbard, which is the archipelago above Norway, and Franz Josef Land in the Russian Arctic and Wrangell Island. And then in the southern summer, they're going to the South Pacific Islands and Atlantic Islands like the Falklands and Antarctica. This is a huge industry.",
  'doc_id': 'Dzlg17y0IMM_131',
  'episode_url': 'https://www.youtube.com/watch?v=Dzlg17y0IMM&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Jonathan Reisman',
  'length': 8595,
  'thumbnail_url': 'https://i.ytimg.com/vi/Dzlg17y0IMM/hq720.jpg',
  'title': 'VEGANS VS MEAT EATERS: Who Will Live Longer & Why You Should Care! | Jonathan Reisman',
  'video_id': 'Dzlg17y0IMM',
  'views': 57838,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.24602526, normalized score: 0.25699127',
  'score': '0.25699127'},
 {'content': "It doesn't necessarily have to mean that it's coming from a negative place or that's why I understand Japanese culture. I don't feel like they mean it maliciously. They literally don't understand. They're so lovely. I consider moving there. But at the same time, I get it. White people tend to have a hard time. Outsiders. They have a real problem with foreigners. Right. Okay. What do they think you're going to do? I have no idea. That I won't speak to. I don't know enough about it. But I know some people that grew up in America that now live there. And they're like, bro, when they had a breakdown of who got the vaccine when, it was like foreigners were in 97th place. It was like every conceivable stack of Japanese person. And then the sickest, oldest foreigner. Then they could finally get it. But it was like your perfectly healthy 19-year-old Japanese kid, 100% was getting the vaccine before you. You know what, there's a weirdness to it.",
  'doc_id': '9I39boHZYjI_205',
  'episode_url': 'https://www.youtube.com/watch?v=9I39boHZYjI&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Sadia Khan',
  'length': 12641,
  'thumbnail_url': 'https://i.ytimg.com/vi/9I39boHZYjI/hqdefault.jpg?sqp=-oaymwEXCJADEOABSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLDMm39lIbI-aBDkCM1MoXAMpAEAkg',
  'title': 'Mastering Sex, Power, Gender Roles, & The Secret to Maintaining Relationships That Last | Sadia Khan',
  'video_id': '9I39boHZYjI',
  'views': 267904,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.24354708, normalized score: 0.24911408',
  'score': '0.24911408'},
 {'content': "I mean, often when you're going to those places, you're doing something physical, let's say hiking or some kind of expedition or mission. And there's a whole bunch of kind of emotionally satisfying and intellectually satisfying and sort of social cohesion things going on when you're in some kind of mission-based plan or on that course of activity. And I think that there's something very satisfying about that. There's very satisfying things about sort of being out in nature where sort of things like geography and climate and weather are important. You know, I'm not in a hospital in the middle of a city where the temperature is controlled. It never rains. I don't even know what the weather's like outside because I haven't looked out a window in some hours. You know, everything smells like disinfectant and all the sounds are beeping and phones ringing and not sort of birds chirping or splashing of animals or whatever. So I think being out in those places is interesting from the natural world standpoint, seeing different ecosystems, different plants and animals.",
  'doc_id': 'Dzlg17y0IMM_110',
  'episode_url': 'https://www.youtube.com/watch?v=Dzlg17y0IMM&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Jonathan Reisman',
  'length': 8595,
  'thumbnail_url': 'https://i.ytimg.com/vi/Dzlg17y0IMM/hq720.jpg',
  'title': 'VEGANS VS MEAT EATERS: Who Will Live Longer & Why You Should Care! | Jonathan Reisman',
  'video_id': 'Dzlg17y0IMM',
  'views': 57838,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.23952556, normalized score: 0.2363312',
  'score': '0.2363312'},
 {'content': "If you hadn't learned it yet, not gonna learn it. Well, I've just graduated high school. I got 45 bucks in my pocket from a job. I got, I got a four handicap. I've just made two holes in ones, 11 days apart last month. I got the best, I'm taking the best looking girl in my school and across town. I've made straight A's so mom and dad are super happy. I got a car that's paid for and I got no curfew. And right out of that, I decided I'm going to go away to Australia to be an exchange student. It was Australia or Sweden. And I remember writing down my 18 year old mind, sunny beaches, Elle McPherson, English speaking, Australia. So I head off to Australia and I'm told by this family that I'm gonna live with we live here. It's a beautiful place. We live on the outskirts of Sydney. You're going to love it. Well, very quickly after I meet this family and three hour drive out of Sydney, which is a bit of an exaggeration of what the outskirts are.",
  'doc_id': 'WQ5HQ6z9uIk_14',
  'episode_url': 'https://www.youtube.com/watch?v=WQ5HQ6z9uIk&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Matthew McConaughey',
  'length': 3773,
  'thumbnail_url': 'https://i.ytimg.com/vi/WQ5HQ6z9uIk/hq720.jpg',
  'title': 'Matthew McConaughey Shares his Trick for Getting What You Want',
  'video_id': 'WQ5HQ6z9uIk',
  'views': 685528,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.23663318, normalized score: 0.22713743',
  'score': '0.22713743'},
 {'content': "I'm guessing as you get into these more remote places that that sort of drops away to next to nothing, or maybe I'm wrong about that. What have you found? Yeah, so it's interesting, you know, if you go from the city to a rural area in America, right? You're sort of moving towards wilderness, but the obesity doesn't drop there for sure. In fact, a lot of obesity in rural America. I would guess it actually goes up, right? So there's a lot of socioeconomics that come into play there. But yeah, so, you know, rural parts of America or other developed countries, you don't see less obesity. You know, there's so much has been mechanized about the way of life that not that physical exertion is the only part. I think nutrition is a big part of it, too, of course. And I think probably other things we don't understand have resulted in. But wait, wouldn't people out in rural America work more, not less? Like physically? Yeah. Well, I mean, so much of it has been mechanized.",
  'doc_id': 'Dzlg17y0IMM_115',
  'episode_url': 'https://www.youtube.com/watch?v=Dzlg17y0IMM&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Jonathan Reisman',
  'length': 8595,
  'thumbnail_url': 'https://i.ytimg.com/vi/Dzlg17y0IMM/hq720.jpg',
  'title': 'VEGANS VS MEAT EATERS: Who Will Live Longer & Why You Should Care! | Jonathan Reisman',
  'video_id': 'Dzlg17y0IMM',
  'views': 57838,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.23475385, normalized score: 0.22116375',
  'score': '0.22116375'},
 {'content': "And several points here, right? The first is that you don't have to disclose, you know, like your membership in this community. For example, when you walk down the street, yeah, some buildings have logos on them, but do you know who's living in what place? Do you have x-ray glasses? Like your augmented reality glasses don't necessarily give you information on who's there, right? People could be Christian, they could be Mormon, whatever, right? All the type of stuff. So number one is these communities don't need to be perceptible or visible to everybody. Not everybody needs to be able to see the map or the full map. Maybe you need to hold an NFT to see some of it. Maybe in some jurisdictions where one node of the network is persecuted, it's invisible, okay? So invisibility is a big part of security, okay? Like stealth, just like a stealth bomber, except it's a stealth, don't get bombed. That's it.",
  'doc_id': 'l4fLax7S2Q0_253',
  'episode_url': 'https://www.youtube.com/watch?v=l4fLax7S2Q0&list=PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo',
  'guest': 'Balaji Srinivasan',
  'length': 16255,
  'thumbnail_url': 'https://i.ytimg.com/vi/l4fLax7S2Q0/hq720.jpg',
  'title': 'EMERGENCY EPISODE: Why This Financial Crisis Is WORSE Than 2008 | Balaji Srinivasan',
  'video_id': 'l4fLax7S2Q0',
  'views': 528422,
  'explainScore': '(vector) [0.13541497 0.03338659 -0.0019899565 0.048069205 0.028292986 0.0010872943 -0.0057838964 -0.083339 -0.036647454 -0.01412428]...  : original score 0.2346412, normalized score: 0.22080567',
  'score': '0.22080567'}]

In [14]:
results = validate_token_threshold(resp, question_answering_prompt, 'what are the effects of drugs on the brain', encoding, 2500, verbose=True)

Token count exceeds token count threshold, reducing size of returned results below token threshold
Total Final Token Count: 2328


In [149]:
from weaviate_interface import WeaviateClient
from llama_index.finetuning import EmbeddingQAFinetuneDataset
from app_features import validate_token_threshold
from tiktoken import get_encoding

encoding = get_encoding("cl100k_base")

#read env vars from local .env file
api_key = os.environ['WEAVIATE_API_KEY']
url = os.environ['WEAVIATE_ENDPOINT']

#instantiate client
client = WeaviateClient(api_key, url, timeout_config=(300,300))

#check if WCS instance is live and ready
client.is_live(), client.is_ready()

(True, True)

In [169]:
gpt = GPT_Turbo(model='gpt-3.5-turbo-0613')

In [107]:
dataset = EmbeddingQAFinetuneDataset.from_json('./data/golden_dataset.json')
queries = list(dataset.queries.values())
len(queries)

250

In [108]:
gpt.get_completion_from_messages('something')

'Hello! How can I assist you today?'

In [83]:
class_name = 'Impact_theory_minilm_256'

In [115]:
# response = client._hybrid_search(query, class_name, display_properties=['content', 'guest', 'summary'], limit=5)
# series = '\n'.join([context_block.format(summary=res['summary'], guest=res['guest'], transcript=res['content']) for res in response]).strip()

In [63]:
prompt = question_answering_prompt_series.format(series=series, question=query)

In [121]:
%%time
total_token_count = 0
answers = []
n = 125
for i, q in enumerate(tqdm(queries[:n])):
    try:
        response = client._hybrid_search(q, class_name, display_properties=['content', 'guest', 'summary'], limit=5)
    except Exception as e:
        print(f'Weaviate failure due to {e}')
        continue
    series = '\n'.join([context_block.format(summary=res['summary'], guest=res['guest'], transcript=res['content']) for res in response]).strip()
    prompt = question_answering_prompt_series.format(series=series, question=q)
    total_token_count += len(encoding.encode(prompt))
    try:
        answer = gpt.get_completion_from_messages(prompt, temperature=0.15)
        answers.append(answer)
    except Exception as e:
        print(f'OpenAI failure due to {e}')
        continue
    if i % 10 == 0:
        print(total_token_count)
        print(answer)
print(f'Total Tokens: {total_token_count} over {n} queries, average of {total_token_count/n} tokens per query.')

  0%|          | 0/125 [00:00<?, ?it/s]

2858
Les Brown's mother reacted by physically assaulting him when she saw him drinking from a white-only water fountain. In the transcript, Les Brown describes the incident where his mother grabbed him by the neck, punched him in the head and face, and got him down on the ground. She then apologized to him, explaining that the water fountains were for white people only and that she would have fought the police officer if he had tried to harm her son.
28994
Tai Lopez suggests that individuals can build the discipline needed to succeed by setting short-term goals and focusing on small wins. He advises against being too ambitious and emphasizes the importance of separating vision from goals. According to Lopez, having a massive vision can lead to burnout and discouragement if one tries to achieve it all at once. Instead, he recommends setting one-day goals and celebrating small victories along the way. This approach helps to build discipline and maintain motivation by providing a sense of

In [143]:
openai.__version__

'1.3.4'

In [144]:
from openai import OpenAI

In [186]:
gpt = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
system_message = 'you are a helpful assistant'
query = 'why did Colin Brady choose Antarctica'

In [188]:
results = client.keyword_search(query, class_name, limit=8, display_properties=['summary', 'content', 'guest'])

In [189]:
series = '\n'.join([context_block.format(summary=res['summary'], guest=res['guest'], transcript=res['content']) for res in results]).strip()
prompt = question_answering_prompt_series.format(series=series, question=query)

In [193]:
len(encoding.encode(prompt))

3891

In [191]:
messages =  [
     {'role': 'system', 'content': system_message},
     {'role': 'assistant', 'content': prompt}
             ]

In [199]:
ans = gpt.chat.completions.create(model='gpt-3.5-turbo-0613', messages=messages, max_tokens=191)

In [203]:
len(encoding.encode(ans.choices[0].message.content))

129

In [204]:
prompt

'\nYour task is to synthesize and reason over a series of transcripts of an interview between Tom Bilyeu and his guest(s).\nAfter your synthesis, use the series of transcripts to answer the below question.  The series will be in the following format:\n\n```\nShow Summary: <summary>\nShow Guest: <guest>\nTranscript: <transcript>\n```\n\n\nStart Series:\n```\nShow Summary: In this episode, Tom Bilyeu interviews Dave Hollis, the COO of The Hollis Co. and former president of theatrical distribution for Disney. Hollis shares his journey of personal growth and development, highlighting his initial skepticism towards self-help and how he overcame it. He discusses the importance of pushing oneself outside of their comfort zone and embracing the possibility of failure in order to grow and find fulfillment. Hollis also emphasizes the significance of congruence between one\'s words and actions, and how he uses the mantra of "stronger" and "better" to motivate himself. He shares his experience wit

Bad pipe message: %s [b'J\x03\x93\x00\x85\x13M\xeb\xeb\x97\x801\xe4\x1a|\xb9\x96h \xb9\xc6\xac\xb9j\xaa\xf6\xb2 w\xf5\xc7\xc1\xf5\xfd\\_Ul\xf7h\x8b\xbb\n\xfa\xbd\x8f\xe8\xd8\xd6\x0b`\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t', b'\x08\x0b\x08\x04\x08\x05\x08']
Bad pipe message: %s [b'\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'\x0b\x7f\x01\xc6\xa5\xec\xa8s\x10\x8b\xb0\x1e\xcck\xef\xd2Z\xaa\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0']
Bad pipe message: %s [b"S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa

In [195]:
4182 - 4097

85